In [2]:
# import moduels
import h5py
import numpy as np
import matplotlib.pylab as plt

from sklearn.model_selection import train_test_split
from sklearn import tree
import pandas as pd
from sklearn.metrics import confusion_matrix

from PIL import Image
import random
import math

import os


In [18]:
# access the data
# this part have to be replaced when the images from the neuronal networks are available
dset = h5py.File("dataset_test.h5","r")

#Access to the input data
RGB = dset["RGB"]
NIR = dset["NIR"]

#Access groundtruth data
GT = dset["GT"]
CLD = dset["CLD"]

print("Thes shape of the images is:")
print(RGB.shape)


# get size and numbers of the dataset
# number of images
n_images = RGB.shape[0]

# imagesize x_direction
sx_image = RGB.shape[1]
#imageszize y_direction
sy_image = RGB.shape[2]

Thes shape of the images is:
(3, 10980, 10980, 3)


In [85]:
# function to create groundtruth

def groundtruth_generator(img_number, x_ind, y_ind,win_x,win_y):
    print(int(x_ind))
    print(y_ind)
    #create temp image
    cout_out = GT[img_number, x_ind:x_ind+win_x,y_ind:y_ind+win_y]
    temp_img = np.where(cout_out==99,3,cout_out)

    #insert the clouds into the temp groundtruth
    temp_cloud = CLD[img_number, x_ind:x_ind+win_x,y_ind:y_ind+win_y]
    
    #cloud_positions = np.where(CLD[img_number] > 10) # check if this number makes sense
    cloud_positions = np.where(temp_cloud > 10) # check if this number makes sense
    temp_img[cloud_positions] = 2
    
    #cout_out = temp_img[x_ind:x_ind+win_x,y_ind:y_ind+win_y]

    final_ground = temp_img.reshape(win_x*win_y,1)
    return final_ground

In [88]:
type(x_coord)

numpy.float64

In [90]:
groundtruth_generator(1,x_coord.astype(np.int64),y_coord.astype(np.int64),s_patch,s_patch)

2681
0


array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=uint8)

In [67]:
#Access the data from the neuronal network part
# path where the files of the networkpart are 
path = 'C:/Users/Pascal/Documents/ETH/Master/HS2021/Image Interpretation/Lab1/brauchbarer_code/output_network'
files = os.listdir(path)
#count how many files there are
num_files = len(files)

#seperate the data into training data and validation data
X_train_n, X_val_n, y_train_n, y_val_n = train_test_split(np.arange(num_files),np.arange(num_files),test_size=0.3,random_state=0)

In [19]:
# TODO apply it in a way that is possible to get the infromation from the name 

# of the ndarray
# set number of images in x and y direction
# number of images in x direction
nx = 5
#number of images in y direction 
ny = 5
# size of patch
# alle sind quadratisch
s_patch = 256
# numbers of categories in the pretrained network Pascal Voc --> 21
n_cat = 21
# compute the coordinates of all top left pixels of each patch
# is for each image the same

x_max = sx_image-s_patch
y_max = sy_image-s_patch 

stepsize_x = math.floor(x_max/nx)
stepsize_y = math.floor(y_max/ny)

#create arrays with the corrdinates
x_coordinates = np.arange(0,x_max,stepsize_x)
y_coordinates = np.arange(0,y_max,stepsize_y)

# create grid
xx , yy = np.meshgrid(x_coordinates,y_coordinates)

In [93]:
# create the X_train etc. arrays
#test = np.load('./output_network/'+str(files[X_train[0]]))
#test.shape

X_train = np.zeros([len(X_train_n)*np.square(s_patch),n_cat])
y_train = np.zeros([len(y_train_n)*np.square(s_patch),1])

X_val = np.zeros([len(X_val_n)*np.square(s_patch),n_cat])
y_val = np.zeros([len(y_val_n)*np.square(s_patch),1])

step_size = np.square(s_patch)

# TODO check if everything is correctly implemented
for i in range(1,len(X_train_n)+1):
    temp = np.load('./output_network/'+str(files[X_train_n[i-1]]))
    X_train[i*step_size-step_size:i*step_size,:] = temp[:-1,:]
    
    x_coord = temp[-1,0].astype(np.int64)
    y_coord = temp[-1,1].astype(np.int64)
    
    y_train[i*step_size-step_size:i*step_size] = groundtruth_generator(1,x_coord,y_coord,s_patch,s_patch)

2681
0
0
5362
8043
5362
5362
5362
2681
8043
8043
8043
5362
8043
0
8043
0
0
2681
2681
8043
0


In [94]:
y_train.shape

(720896, 1)

# Classifier part

In [95]:
# train classifier
dt_clf = tree.DecisionTreeClassifier(max_depth=5)
dt_clf.fit(X_train, y_train)
#dt_clf.score(X_val, y_val)

DecisionTreeClassifier(max_depth=5)

In [ ]:
# confusion matrix --> test set have to be created

# test classifier and create confusion matrix
#y_pred = dt_clf.predict(X_test)
#conf_matrix = confusion_matrix(y_test, y_pred)
#print(conf_matrix)